In [1]:
import pandas as pd
import numpy as np
import os
os.listdir('../my/data')

['BC_1022.csv',
 'BC_32ea_k100_cyto.csv',
 'BC_32ea_k15_cyto.csv',
 'K15_1022.csv',
 'K15_common.csv',
 'normal_1022.csv',
 'Normal_21ea_k100_cyto.csv',
 'Normal_21ea_k15_cyto.csv',
 'PC_1022.csv',
 'PC_20ea_k100_cyto.csv',
 'PC_20ea_k15_cyto.csv',
 'RC_20ea_k100_cyto.csv',
 'TCGA_common.csv',
 'TCGA_data.csv']

In [7]:
#NA
tcga = pd.read_csv('../my/data/TCGA_common.csv')
urine = pd.read_csv('../my/data/K15_common.csv')

urine_BC_Norm=urine.loc[urine['cancer'].isin(['Normal','BC'])]
tcga_BC_Norm=tcga.loc[tcga['y'].isin(['Normal','BC'])]
X_train = tcga_BC_Norm.drop('y', axis = 1)
y_train = tcga_BC_Norm['y']
X_test = urine_BC_Norm.drop('cancer',axis=1)
y_test= urine_BC_Norm['cancer']

print('X_train shape : , ', X_train.shape, 'y_train : ', y_train.shape)
print('X_test shape : , ', X_test.shape, 'y_test : ', y_test.shape)

X_train shape : ,  (2114, 763) y_train :  (2114,)
X_test shape : ,  (53, 763) y_test :  (53,)


In [8]:
y_train[y_train == 'BC'] =1
y_train[y_train == 'Normal'] = 0

y_test[y_test == 'BC'] = 1
y_test[y_test == 'Normal'] = 0

y_train = y_train.astype('int')
y_test = y_test.astype('int')

C:\Users\wjssm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\wjssm\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\wjssm\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\wjssm\Anaconda3\lib\site-packages\ipykernel_launcher

In [9]:
y_train.value_counts()

0    1706
1     408
Name: y, dtype: int64

In [10]:
y_test.value_counts()

1    32
0    21
Name: cancer, dtype: int64

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve

def get_eval(y_test, pred) :
    
    print('accuracy : {0:.4f} precision : {1:.4f} recall : {2:.4f} F1 : {3:.4f} AUC : {4:.4f}'.format(
                                                        accuracy_score(y_test, pred), 
                                                        precision_score(y_test, pred),
                                                        recall_score(y_test, pred),
                                                        f1_score(y_test,pred),
                                                        roc_auc_score(y_test, pred)
                                                                      ))
    
    
    
    #print('accuracy : {0:.4f}'.format(accuracy_score(y_test,pred)), 'roc_auc_score : ', roc_auc_score(y_test,pred))
    #print(classification_report(y_test, pred))
    print(confusion_matrix(y_test, pred))

In [13]:
import matplotlib.pyplot as plt

#draw roc curve
def roc_curve_plot(y_test, pred_proba) :
    
    fpr, tpr, threshold = roc_curve(y_test, pred_proba)
    plt.plot(fpr, tpr, label = 'ROC')
    plt.plot([0,1],[0,1],'k--', label = 'Random')
    
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1)
    plt.ylim(0,1)
    
    plt.xlabel('FPR(1-Sensitivity)')
    plt.ylabel('TPR(Recall)')
    plt.legend()

# scaling

In [14]:
X_test = X_test[X_train.columns]
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler
#MinMaxScaler
X_train_mms = MinMaxScaler().fit_transform(X_train)
X_test_mms = MinMaxScaler().fit_transform(X_test)

#MaxAbsScaler
X_train_mas = MaxAbsScaler().fit_transform(X_train)
X_test_mas = MaxAbsScaler().fit_transform(X_test)

In [24]:
import warnings
warnings.filterwarnings('ignore')

# voting

In [32]:
from sklearn.ensemble import VotingClassifier

def voting_func(models, X_train, X_test) :

    soft_vote = VotingClassifier(estimators = models, voting = 'soft')
    hard_vote = VotingClassifier(estimators = models, voting = 'hard')

    print('soft voting')
    soft_vote.fit(X_train ,y_train)
    vote_pred = soft_vote.predict(X_test)
    get_eval(y_test, vote_pred)

    print('hard voting')
    hard_vote.fit(X_train, y_train)
    vote_pred = hard_vote.predict(X_test)
    get_eval(y_test, vote_pred)

## Original data

In [26]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 156, max_depth = 8)
dt.fit(X_train, y_train)

#RandomForest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, max_depth=7, min_samples_leaf=8, min_samples_split=16,
                                n_estimators = 200)
rf.fit(X_train, y_train)

from sklearn.svm import SVC
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train, y_train)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [35]:
voting_func([('DT', dt), ('RF', rf), ('SVM',svc), ('LR', lr), ('GB', gb)], X_train, X_test)

soft voting
accuracy : 0.6038 precision : 0.6222 recall : 0.8750 F1 : 0.7273 AUC : 0.5327
[[ 4 17]
 [ 4 28]]
hard voting
accuracy : 0.5849 precision : 0.6042 recall : 0.9062 F1 : 0.7250 AUC : 0.5007
[[ 2 19]
 [ 3 29]]


## MinMaxScale

In [36]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 156, max_depth = 8)
dt.fit(X_train_mms, y_train)

#RandomForest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, max_depth=7, min_samples_leaf=8, min_samples_split=16,
                                n_estimators = 200)
rf.fit(X_train_mms, y_train)

from sklearn.svm import SVC
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train_mms, y_train)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_mms, y_train)

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(X_train_mms, y_train)

voting_func([('DT', dt), ('RF', rf), ('SVM',svc), ('LR', lr), ('GB', gb)], X_train_mms, X_test_mms)

soft voting
accuracy : 0.6604 precision : 0.7692 recall : 0.6250 F1 : 0.6897 AUC : 0.6696
[[15  6]
 [12 20]]
hard voting
accuracy : 0.5849 precision : 0.6136 recall : 0.8438 F1 : 0.7105 AUC : 0.5171
[[ 4 17]
 [ 5 27]]


## MaxAbsScale

In [ ]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 156, max_depth = 8)
dt.fit(X_train_mas, y_train)

#RandomForest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, max_depth=7, min_samples_leaf=8, min_samples_split=16,
                                n_estimators = 200)
rf.fit(X_train_mas, y_train)

from sklearn.svm import SVC
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train_mas, y_train)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_mas, y_train)

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(X_train_mas, y_train)

voting_func([('DT', dt), ('RF', rf), ('SVM',svc), ('LR', lr), ('GB', gb)], X_train_mas, X_test_mas)

soft voting
